### COCO dataset cars

воспользуйтесь инструкцией с сайта https://cocodataset.org/#home


In [1]:
# чтобы скачать часть coco dataset нужно установить библиотеку
%pip install pycocotools

In [ ]:
#loading pictures and make boxes
#запускать только если нужно загрузить датасет
from pycocotools.coco import COCO
import requests
from glob import glob

def load(images, num = 0):
    i = num
    try:
        for img in images[num:]:
            i+=1
            if i%100 == 0:
                print(i)
            data = requests.get(img['coco_url']).content
            with open("images/" + img['file_name'], "wb") as f:
                f.write(data)

            with open("labels/" + img['file_name'].split(".")[0] + ".txt", "w") as f:
                f.write("")
    except:
        load(images, i)
    
def make_boxes(annotations, images, class_num):
    f_names = [name.split("/")[-1] for name in glob("images/COCO*")]
    for ann in annotations:
        bbox = ann["bbox"]
        img_info = coco.loadImgs(ann["image_id"])
        category_id = ann["category_id"]
        if(len(img_info) > 1):
            print("ERROR")
            sys.exit()
        if(img_info[0]["file_name"] not in f_names):
            continue
        
        filename = img_info[0]["file_name"].split(".")[0] + ".txt"
        
        width = float(img_info[0]["width"])
        height = float(img_info[0]['height'])
        xmin = bbox[0]
        ymin = bbox[1]
        xmax = min(xmin + bbox[2], width)
        ymax = min(ymin + bbox[3], height)

        x = (xmax + xmin)/(2*width)
        y = (ymax + ymin)/(2*height)
        w = (xmax - xmin)/width
        h = (ymax - ymin)/height

        with open("labels/" + filename, "a") as f:
            f.write(f"{class_num[category_id]} {x} {y} {w} {h}\n")
            
            
# coco = COCO("путь до COCO/annotations/instances_train2014.json")

for i, name in  enumerate(['car', 'motorcycle', 'bus', 'train', 'truck']):
    catId = coco.getCatIds(catNms = [name])
    imgIds = coco.getImgIds(catIds = catId)
    images = coco.loadImgs(imgIds)
    load(images, 0)

categories = ['car', 'motorcycle', 'bus', 'train', 'truck']
catId = coco.getCatIds(catNms = categories)

imgIds = coco.getImgIds(catIds = catId)
images = coco.loadImgs(imgIds)

annIds = coco.getAnnIds(catIds = catId)
annotations = coco.loadAnns(annIds)

id_nums = dict(zip(catId, range(len(catId))))

make_boxes(annotations, images, id_nums)   

print("новые метки классов:\n", dict(zip(categories, range(len(catId)))))    

новые метки классов:
 {'car': 0, 'motorcycle': 1, 'bus': 2, 'train': 3, 'truck': 4}


In [ ]:
#train_val preparation
import numpy as np
import subprocess

img_names = [t.split('/')[-1] for t in glob("images/COCO*")]
label_names = [t.split('.')[0] + ".txt" for t in img_names]

ind = np.arange(len(img_names))
rand_ind = np.random.choice(ind, size = int(0.03 * len(ind)))
val_img = [img_names[i] for i in rand_ind]
val_labels = [label_names[i] for i in rand_ind]

for img in val_img:
    subprocess.run(["mv",'images/' +img, "./../val/images/" + img])
    
for label in val_labels:
    subprocess.call(["mv",'labels/' + label, "./../val/labels/" + label])

## тренировка модели

In [ ]:
import os
os.chdir("")

In [ ]:
# yaml файл указывающий на датасет
dataset_path = "путь к датасету"
with open("./object_detection/yolov7/data/car.yaml", "w") as f:
    f.write(f"train: {dataset_path}/train\nval: {dataset_path}/val/\nnc: 2\nnames: [\"num_palette\", \"car\"]")

In [145]:
!python3 train.py --img 640 --workers 8 --batch 16 --epochs 200 --data data/car.yaml --weights yolov7.pt --cfg cfg/training/yolov7.yaml --device 2 << 3

/bin/bash: warning: here-document at line 0 delimited by end-of-file (wanted `3')
^C
Traceback (most recent call last):
  File "train.py", line 24, in <module>
    import test  # import test.py to get mAP after each epoch
  File "/home/latypov/YOLOv7/yolov7/test.py", line 12, in <module>
    from models.experimental import attempt_load
  File "/home/latypov/YOLOv7/yolov7/models/experimental.py", line 6, in <module>
    from models.common import Conv, DWConv
  File "/home/latypov/YOLOv7/yolov7/models/common.py", line 17, in <module>
    from utils.plots import color_list, plot_one_box
  File "/home/latypov/YOLOv7/yolov7/utils/plots.py", line 15, in <module>
    import seaborn as sns
  File "/usr/local/lib/python3.6/site-packages/seaborn/__init__.py", line 2, in <module>
    from .rcmod import *  # noqa: F401,F403
  File "/usr/local/lib/python3.6/site-packages/seaborn/rcmod.py", line 7, in <module>
    from . import palettes
  File "/usr/local/lib/python3.6/site-packages/seaborn/palettes